In [27]:
"""
A Keras model for sem seg on PV solar
"""
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, Cropping2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model, to_categorical
from keras.optimizers import Adam
#from FCN_utils import *
from keras.metrics import categorical_accuracy
from keras.models import model_from_json

#from sklearn.metrics import confusion_matrix

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt

import json
import glob
import gc

#https://stats.stackexchange.com/questions/285255/nns-multiple-sigmoid-binary-cross-entropy-giving-better-results-than-softmax
#Try with binary crossentropy, cateogrical_accuracy

In [6]:
INPUT_SHAPE = (200,200,14) 
BATCH_SIZE = 12
#N_CLASSES = 7
MAX_ITERATIONS = 100
LEARNING_RATE=5e-7


In [7]:
""" get training data """

all_files = glob.glob('./trn_data/*')
#print (all_files)

m = len(all_files)

a = np.load(all_files[0])
print (a.files)
print (a['data'].shape)

all_data = np.zeros((m,*a['data'].shape))
all_annotations = np.zeros((m,*a['annotation'].shape))

print (all_data.shape)

ii=0
for f in all_files:
    inp = np.load(f)
    all_data[ii,:,:,:] = inp['data']
    all_annotations[ii,:,:] = inp['annotation']
    ii+=1



print (np.unique(a['annotation']))
print (np.unique(all_annotations))
all_annotations[all_annotations==64.]=0
all_annotations[all_annotations==96.]=0
all_annotations[all_annotations==255]=1

print (np.unique(all_annotations))
print ((a['data']/10000.).clip(0.0,1.0))


['./trn_data/227.npz', './trn_data/77.npz', './trn_data/189.npz', './trn_data/2.npz', './trn_data/67.npz', './trn_data/74.npz', './trn_data/52.npz', './trn_data/103.npz', './trn_data/397.npz', './trn_data/0.npz', './trn_data/380.npz', './trn_data/196.npz', './trn_data/423.npz', './trn_data/222.npz', './trn_data/355.npz', './trn_data/157.npz', './trn_data/348.npz', './trn_data/96.npz', './trn_data/267.npz', './trn_data/249.npz', './trn_data/36.npz', './trn_data/435.npz', './trn_data/288.npz', './trn_data/332.npz', './trn_data/350.npz', './trn_data/399.npz', './trn_data/392.npz', './trn_data/40.npz', './trn_data/353.npz', './trn_data/11.npz', './trn_data/322.npz', './trn_data/251.npz', './trn_data/333.npz', './trn_data/314.npz', './trn_data/133.npz', './trn_data/378.npz', './trn_data/273.npz', './trn_data/60.npz', './trn_data/309.npz', './trn_data/319.npz', './trn_data/20.npz', './trn_data/182.npz', './trn_data/419.npz', './trn_data/163.npz', './trn_data/147.npz', './trn_data/172.npz', '

[0]
[  0. 255.]
[0. 1.]
[[[0.0709 0.1013 0.0937 ... 0.0012 0.1166 1.    ]
  [0.0514 0.0881 0.0906 ... 0.0012 0.1166 1.    ]
  [0.049  0.0904 0.0884 ... 0.0012 0.1166 1.    ]
  ...
  [0.056  0.0852 0.0866 ... 0.0012 0.1245 1.    ]
  [0.0516 0.0793 0.0833 ... 0.0012 0.1228 1.    ]
  [0.0666 0.0871 0.0908 ... 0.0012 0.1228 1.    ]]

 [[0.0744 0.1085 0.0967 ... 0.0012 0.1166 1.    ]
  [0.0514 0.0895 0.0907 ... 0.0012 0.1166 1.    ]
  [0.0485 0.089  0.0874 ... 0.0012 0.1166 1.    ]
  ...
  [0.054  0.0869 0.0904 ... 0.0012 0.1245 1.    ]
  [0.0493 0.0782 0.0874 ... 0.0012 0.1228 1.    ]
  [0.0447 0.0754 0.0851 ... 0.0012 0.1228 1.    ]]

 [[0.0832 0.1155 0.0982 ... 0.0012 0.1166 1.    ]
  [0.0577 0.0931 0.0909 ... 0.0012 0.1166 1.    ]
  [0.0458 0.0842 0.0834 ... 0.0012 0.1166 1.    ]
  ...
  [0.0523 0.0891 0.0904 ... 0.0012 0.1245 1.    ]
  [0.0558 0.0898 0.0962 ... 0.0012 0.1228 1.    ]
  [0.056  0.0901 0.0917 ... 0.0012 0.1228 1.    ]]

 ...

 [[0.0765 0.105  0.1077 ... 0.0011 0.1327 1.  

In [8]:

train_images = (all_data/10000.).clip(0.0,1.0)

train_annotations = to_categorical(all_annotations)
N_CLASSES = train_annotations.shape[-1]

print (N_CLASSES)

#train_annotations = to_categorical(train_annotations, num_classes=N_CLASSES)
#N_CLASSES=train_annotations.shape[-1]



2


In [32]:
def save_image(arr, fname):
    #print (arr, arr.shape)
    #arr = arr.astype('uint8')
    if (len(arr.shape)>2):
        gamma = 2.5
        
        im = Image.fromarray((arr*gamma*255.).clip(0,255).astype('uint8'))
        im.save(fname)
    else:
        #im should have scalars in 
        im = Image.fromarray((arr*40).astype('uint8'))
        #plt.imshow(im)
        #plt.show()
        im.save(fname)

In [24]:
train_images_sample = train_images[60:100,:,:,:]
train_annotations_sample = train_annotations[60:100,:,:,:]

In [25]:
def IoU(prediction, annotation):
    #prediciton and annotation should have the same shape: (m,w,l,n_c)
    n_c = annotation.shape[-1]
    prediction = np.argmax(prediction, axis=-1)
    annotation = np.argmax(annotation, axis=-1)
    intersections = {}
    unions = {}
    for c in range(n_c):
        prediction_c = prediction==c
        annotation_c = annotation==c
        #intersection = AND(a,b) = a*b
        #union = OR(a,b) = a+b
        intersection_c = prediction_c*annotation_c
        union_c = prediction_c+annotation_c
        intersections[c] = np.sum(intersection_c)
        unions[c] = np.sum(union_c)
    return intersections, unions

In [14]:
model_json_files = glob.glob('*.json')
print (model_json_files)

['model_unet.json', 'model_deep_resunet.json', 'model_segnet.json', 'model_resunet.json']


In [ ]:
model_batch_size = {
    'model_unet':16, 
    'model_deep_resunet':16,
    'model_segnet':32,
    'model_resunet':12,
    
}




for model_json_file in model_json_files:
    
    model_str = model_json_file.split('.')[0]
    print (model_str)
    json_file = open(model_json_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    
    
    opt = Adam(lr=1e-5)
    model.compile(optimizer=opt, loss='binary_crossentropy',metrics=[categorical_accuracy])
    print (model.summary())
    
    history = model.fit(x=train_images, y=train_annotations, validation_split = 0.181818, verbose=1, batch_size=model_batch_size[model_str], epochs=400)
    
    print (history.history)
    epochs = np.arange(len(history.history['val_loss']))
    
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.plot(epochs, history.history['loss'], label='loss', color='red')
    ax1.plot(epochs, history.history['val_loss'], label='val_loss', color='orange')
    ax1.set_ylabel('loss')

    ax2 = ax1.twinx()
    ax2.plot(epochs, history.history['categorical_accuracy'], label='cat_accuracy', color='blue')
    ax2.plot(epochs, history.history['val_categorical_accuracy'], label='val_cat_accuracy', color='green')
    fig.legend(loc='upper left')

    
    prediction = model.predict(x=train_images_sample)
    
    intersections, unions = IoU(prediction,train_annotations_sample)
    
    fig.text(0,0,'IoU: {0:.2f}'.format(round(intersections[1]/unions[1],2)))
    

    plt.savefig(model_str+'.png')
    plt.close()
    
    
    for i in range(40):
        gamma = 2.5
        save_image(np.argmax(prediction[i,:,:,:], axis=2), 'sample_out/'+model_str+'_'+str(i)+'_PRED.png')
        save_image(train_images_sample[i,:,:,0:3], 'sample_out/'+model_str+'_'+str(i)+'_INPUT.png')
        save_image(np.argmax(train_annotations_sample[i,:,:,:], axis=2), 'sample_out/'+model_str+'_'+str(i)+'_TRUE.png')
     
    
    
    
    
    model.save(model_str+'.h5')  # creates a HDF5 file 'my_model.h5'
    del model
    K.clear_session()
    gc.collect()

    

model_unet
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 14) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 200, 200, 64) 8128        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 200, 200, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 100, 100, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________

Train on 360 samples, validate on 80 samples
Epoch 1/400
360/360 [==============================] - 39s 108ms/step - loss: 0.2097 - categorical_accuracy: 0.9733 - val_loss: 0.2033 - val_categorical_accuracy: 0.9537
Epoch 2/400
360/360 [==============================] - 36s 99ms/step - loss: 0.1283 - categorical_accuracy: 0.9757 - val_loss: 0.1891 - val_categorical_accuracy: 0.9537
Epoch 3/400
360/360 [==============================] - 36s 100ms/step - loss: 0.1208 - categorical_accuracy: 0.9757 - val_loss: 0.1844 - val_categorical_accuracy: 0.9537
Epoch 4/400
360/360 [==============================] - 36s 100ms/step - loss: 0.1171 - categorical_accuracy: 0.9757 - val_loss: 0.1796 - val_categorical_accuracy: 0.9537
Epoch 5/400
 32/360 [=>............................] - ETA: 30s - loss: 0.0898 - categorical_accuracy: 0.9835

In [ ]:
intersections, unions = IoU(prediction,train_annotations_sample)

for c,v in intersections.items():
    print ('for class ',c,' IoU was ',float(v)/unions[c])